In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import seaborn.objects as so

In [2]:
directory_path_box_max = r'../../models/Box/3/max'
directory_path_box_min = r'../../models/Box/3/min'

In [7]:
directory_path_min = r'../../../models/KMeans/output/q3_c3/q3_c3_top_10_normal2024-04-30 04:57:39.parquet'
directory_path_max = r'../../../models/KMeans/output/q3_c3/q3_c3_top_10_normal2024-04-30 04:57:39.parquet'
df_20_col = pd.read_parquet('../../../models/KMeans/output/seatunnal_20col.parquet')
df_original = pd.read_parquet('../../../Sonar/seatunnel_all_information.parquet')

In [8]:
df_min = pd.read_parquet(directory_path_min)
df_max = pd.read_parquet(directory_path_max)

In [9]:
df_min.columns

Index(['col', 'cluter0_q3', 'cluter0_q1', 'shape_c0_q3', 'shape_c0_q1',
       'cv_q3_c0', 'cv_q1_c0', 'label', 'median_0', 'cluter2_q3', 'cluter2_q1',
       'shape_c2_q3', 'shape_c2_q1', 'cv_q3_c2', 'cv_q1_c2', 'median_2',
       'cluter1_q3', 'cluter1_q1', 'shape_c1_q3', 'shape_c1_q1', 'cv_q3_c1',
       'cv_q1_c1', 'median_1', 'Q3', 'Q1', 'max_q3_equa', 'min_q3_equa',
       'max_q1_equa', 'min_q1_equa'],
      dtype='object')

เรียงลำดับมากไปน้อย q3

In [10]:
df_max_sort = df_max[
    ['col', 'cluter0_q3', 'cluter1_q3', 'cluter2_q3', 'cluter0_q1', 'cluter1_q1', 'cluter2_q1', 'label']]

In [11]:
df_max_sort

,col,cluter0_q3,cluter1_q3,cluter2_q3,cluter0_q1,cluter1_q1,cluter2_q1,label
17240,"[D_change, C_change, D_percent, B_percent, CP_...",[78.63486111111112],[19.405555555555555],[348.2376388888889],[2.944722222222222],[1.98875],[1.7270833333333333],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
19,"[D_change, C_percent]",[31.253888888888888],[78.63486111111112],[308.30875000000003],[2.4277777777777776],[2.912013888888889],[1.8347222222222221],"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
154,"[D_change, D_percent, C_percent]",[78.63486111111112],[31.253888888888888],[308.30875000000003],[2.912013888888889],[2.4277777777777776],[1.8347222222222221],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2688,"[D_change, CP_change, D_percent, B_percent, OO...",[299.3045833333333],[23.01451388888889],[80.51416666666667],[1.437638888888889],[2.1056944444444445],[2.9475],"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
18,"[D_change, D_percent]",[75.51215277777779],[0.4582638888888889],[267.04340277777777],[2.8247916666666666],[0.2553472222222222],[5.258194444444444],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
80004,"[begin_D, begin_B, begin_CP, end_CP, end_C, D_...",[90.09361111111112],[37.306805555555556],[156.89493055555556],[7.331111111111111],[1.5184722222222222],[4.976527777777778],"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
107720,"[begin_B, begin_CP, begin_C, begin_OOA, end_D,...",[156.89493055555556],[37.76444444444444],[90.80916666666667],[4.435555555555555],[1.443888888888889],[7.982361111111111],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
236,"[begin_D, end_D, D_change, OOA_change]",[35.09194444444445],[146.78201388888888],[70.74222222222222],[1.2305555555555556],[6.529097222222222],[5.293055555555555],"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2953,"[begin_D, begin_B, end_D, D_change, CP_change,...",[35.646388888888886],[146.78201388888888],[71.35743055555555],[1.2914583333333334],[5.896527777777778],[5.4065972222222225],"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3250,"[begin_D, begin_CP, end_D, D_change, B_change,...",[71.35743055555555],[35.646388888888886],[146.78201388888888],[5.4065972222222225],[1.2914583333333334],[5.896527777777778],"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


loop sort values of q3 minimux

In [13]:
q3 = df_max_sort.iloc[:, [1, 2, 3]]
q3

,cluter0_q3,cluter1_q3,cluter2_q3
17240,[78.63486111111112],[19.405555555555555],[348.2376388888889]
19,[31.253888888888888],[78.63486111111112],[308.30875000000003]
154,[78.63486111111112],[31.253888888888888],[308.30875000000003]
2688,[299.3045833333333],[23.01451388888889],[80.51416666666667]
18,[75.51215277777779],[0.4582638888888889],[267.04340277777777]
80004,[90.09361111111112],[37.306805555555556],[156.89493055555556]
107720,[156.89493055555556],[37.76444444444444],[90.80916666666667]
236,[35.09194444444445],[146.78201388888888],[70.74222222222222]
2953,[35.646388888888886],[146.78201388888888],[71.35743055555555]
3250,[71.35743055555555],[35.646388888888886],[146.78201388888888]


In [ ]:
q1 = df_max_sort.iloc[:, [4, 5, 6]]

In [ ]:
def loop_check_value_q(df):
    index_list = []
    min_values_list = []
    max_values_list = []
    for i in range(len(df)):
        x = df.iloc[i].sort_values()
        print('index:', df.index[i])
        # print('value:', x)
        min = x[0]
        max = x[2]
        print('min:', x[0])
        print('max:', x[2])
        index_list.append(df.index[i])
        min_values_list.append(min)
        max_values_list.append(max)
    
    return index_list, min_values_list, max_values_list

In [ ]:
index, min_q3 , max_q3 = loop_check_value_q(q3)

In [ ]:
min_q3

In [ ]:
max_q3

min and max value of q1

In [ ]:
index, min_q1 , max_q1 = loop_check_value_q(q1)

min and max value of q3

In [ ]:
concat_col = pd.concat([pd.DataFrame(index, columns=['index']), pd.DataFrame(min_q1, columns=['min']), pd.DataFrame(max_q3, columns=['max'])], axis=1) 

In [ ]:
concat_col = pd.DataFrame(concat_col)
concat_col.set_index('index', inplace=True)
concat_col

In [ ]:
df_max_sort

In [ ]:
df_max_class = pd.merge(df_max_sort, concat_col, left_index=True, right_index=True)

In [ ]:
df_max_class

In [ ]:
df_max_class.to_parquet('../../models/KMeans/output/q3_c3/df_max_time_class.parquet')

Model classification of q3 SVM

In [ ]:
for i in df_max_class.iloc:
    # print("I ::", i)
    
    df_compare = df_20_col[list(i['col'])]
    # print("DF ::", df_compare)
    
    arr = np.array(i['label'])
    # print("Arr ::", arr)
    
    df_arr = pd.DataFrame(arr.reshape(-1,1), columns=['label'])
    # print("DF Arr ::", df_arr)
    
    df_col_combined = pd.concat([df_compare, df_arr], axis=1)
    df_col_combined = pd.concat([df_col_combined, df_original['total_time']], axis=1)
    
    date = pd.to_timedelta(df_col_combined['total_time'])
    time_hours = date.dt.total_seconds() / 3600

    df_col_combined['hours'] = time_hours
    df_col_combined['min'] = [i['min']] * len(df_col_combined)
    df_col_combined['max'] = [i['max']] * len(df_col_combined)
    
    print("DF Combined ::", df_col_combined)